In [1]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtWidgets import QApplication, QMainWindow, QDesktopWidget, QTableWidget, QTableWidgetItem, QLineEdit, \
    QGridLayout, QLabel, QInputDialog, QMessageBox
import random
from treelib import Tree
import numpy as np
import plotly
import plotly.graph_objs as go
import pandas as p
def sumdep(a,koldep):
    rezdep=0
    for i in range(len(a)):
        rezdep += koldep[a[i]]
    return rezdep
def rezuzg(a):
    rez = 1
    for o in range(len(a)):
            for k in range(o+1,len(a)):
                    rez *= uzg[a[o]][a[k]]
    return rez
def EndAlg(a,SumDep,PrUzg,koldep):
    end = 0
    record = 0
    if sumdep(a,koldep) > SumDep*PrUzg:
            record = rezuzg(a)
            end = 1
    return end, record
def sumdepmgtm(a,koldep):
    rezdep=0
    for i in range(len(a)):
        rezdep = rezdep + koldep[a[i]-1]
    return rezdep
def rezuzgmgtm(a):
    rez = 1
    for o in range(len(a)):
            for k in range(o+1,len(a)):
                    rez = rez * uzg[a[o]-1][a[k]-1]
    return rez
def SplitIntSort(s):
    s=s.split('-')
    for i in range(len(s)):
        s[i] = int(s[i])
    s=sorted(s)
    return s
def inversetoarr(arr,n):
    indexes = []
    for i in range(n):
        if arr[i] == 1:
            indexes.append(i)
    return indexes
def autbriding(a,n):
    max = 0
    min = 999
    maxindex = -1
    minindex = -1
    for i in range(len(a)):
        if a[i][n] > max:
            max = a[i][n]
            maxindex = i
        if a[i][n] < min:
            min = a[i][n]
            minindex = i
    return maxindex,minindex
def inbriding(a,n):
    max1 = 0
    max1index = -1
    max2index = -1
    for i in range(len(a)):
        if a[i][n] > max1:
            max1 = a[i][n]
            max1index = i
    rizn = 999
    for i in range(len(a)):
        if max1 - a[i][n] < rizn and i != max1index:
            rizn = max1 - a[i][n]
            max2index = i
    return max1index,max2index
def jadniyalg(n,SumDep,PrUzg,koldep):
    a = []
    max = 0
    for i in range(n):
        for j in range(i,n):
            rez = uzg[i][j]
            if rez>max:
                max = rez
                a = [i,j]
    EndAlg(a,SumDep,PrUzg,koldep)
    max = 0
    btempmax=[]
    b = []
    record = -1
    for i in range(n-2):
        for j in range(0,n):
            if j not in a:
                b=a.copy()
                b.append(j)
                b=sorted(b)
                rez = rezuzg(b)
                if rez>max:
                    max = rez
                    btempmax = b.copy()
                b.remove(j)
        a=btempmax.copy()
        btempmax.clear()
        max = 0
        end, record = EndAlg(a,SumDep,PrUzg,koldep)
        if end == 1:
            return record,a,sumdep(a,koldep)
def mgtmalg(n,SumDep,PrUzg,koldep):
    L=[]
    record=0
    recordID = ""
    ftree = Tree()
    ftree.create_node("", "root")
    for i in range(1,n+1):
            ftree.create_node(str(i),str(i),parent="root",data=1)
            L.append(str(i))
    while(True):
        maxuzg=0
        identifier = ""
        K=L.copy()
        for i in range(len(L)):
            if ftree.get_node(L[i]).data < record:
                K.remove(L[i])
        L.clear()
        L=K.copy()
        for i in range(len(L)):
            if maxuzg < ftree.get_node(L[i]).data:
                maxuzg = ftree.get_node(L[i]).data
                identifier = ftree.get_node(L[i]).identifier
        ParseArr=SplitIntSort(identifier)
        L.remove(identifier)
        if sumdepmgtm(ParseArr,koldep) > SumDep*PrUzg:
                if rezuzgmgtm(ParseArr) > record:
                    record = rezuzgmgtm(ParseArr)
                    recordID = identifier
        else:
            for i in range(1,n+1):
                ParseID = SplitIntSort(identifier)
                if i not in ParseID:
                    path = identifier+"-"+str(i)
                    ParseArr = SplitIntSort(path)
                    ftree.create_node(str(i),path,parent=identifier
                                  ,data=rezuzgmgtm(ParseArr))
                    L.append(path)
        if len(L) == 0: break
    RecordArr = SplitIntSort(recordID)
    return record,RecordArr,sumdepmgtm(RecordArr,koldep)
def GA(n,SumDep,PrUzg,operator,N,koldep):
    a=[]
    osob=[]
    pmutacii = 0.3
    indexrepeat = 0
    Record = 0
    while indexrepeat < N:
        while len(osob) < 10:
            a.clear()
            for j in range(n):
                a.append(random.randrange(0, 2, 1))
            a.append(rezuzg(inversetoarr(a,n)))
            if sumdep(inversetoarr(a,n),koldep) > SumDep*PrUzg and a not in osob:
                osob.append(a.copy())
        max = 0
        for i in range(10):
            if osob[i][n] > max:
                max = osob[i][n]
        if max > Record:
            Record = max
            indexrepeat = 0
        else:
            indexrepeat += 1
        if operator == 1:
            father1,father2 = inbriding(osob,n)
        if operator == 2:
            father1,father2 = autbriding(osob,n)
        randomkrossingover = [random.randrange(0, n, 1),random.randrange(0, n, 1)]
        randomkrossingover = sorted(randomkrossingover)
        nashadok = [osob[father1].copy(),osob[father2].copy()]
        if randomkrossingover[0] == randomkrossingover[1]:
            nashadok[0][randomkrossingover[0]],nashadok[1][randomkrossingover[0]] = nashadok[1][randomkrossingover[0]],nashadok[0][randomkrossingover[0]]
        else:
            for i in range(randomkrossingover[0]+1,randomkrossingover[1]):
                nashadok[0][i],nashadok[1][i] = nashadok[1][i],nashadok[0][i]
        for i in range(2):
            if random.random() > pmutacii:
                indexmutacii = random.randrange(0,n,1)
                if nashadok[i][indexmutacii] == 1:
                    nashadok[i][indexmutacii] = 0
                else: nashadok[i][indexmutacii] = 1
        for i in range(2):
            for j in range(n):
                if sumdep(inversetoarr(nashadok[i],n),koldep) <= SumDep*PrUzg:
                    if nashadok[i][j] == 0:
                        nashadok[i][j] = 1
                else: break
        for i in range(2):
            nashadok[i][n] = rezuzg(inversetoarr(nashadok[i],n))
        for i in range(2):
            min = 999
            minindex = -1
            for j in range(10):
                if osob[j][n] < min:
                    min = osob[j][n]
                    minindex = j
            if nashadok[i][n] > min:
                if nashadok[i] not in osob:
                    osob[minindex] = nashadok[i].copy()
    recordosob = 0
    for i in range(len(osob)):
        if osob[i][n] == Record:
            recordosob = osob[i]
    recordosob = inversetoarr(recordosob,n)
    return Record,recordosob,sumdep(recordosob,koldep)
def randomdata(n,koldep,uzg,d1,d2,m1,m2):
    for i in range(n):
        koldep.append(random.randint(m2-d2,m2+d2))
        for j in range(i+1,n):
            uzg[i][j] = random.uniform(m1-d1,m1+d1)
def Start(n):
        uzg.clear()
        koldep.clear()
        SumDep = 0
        for o in range(n):
            uzg.append([])
            for k in range(n):
                uzg[o].append(0)
        return SumDep
def GenerTask(n,d1,d2):
    SumDep = Start(n)
    randomdata(n,koldep,uzg,d1,d2,m1,m2)
    for l in range(n):
        SumDep += koldep[l]
    return SumDep,koldep
    
m1 = 0.5
m2 = 80
PrUzg = 0.5
uzg = []
koldep = []
SumDep = 0
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 240)
        self.koldep = []
        self.n = -1
        self.SumDep = 0
        self.N = 50
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.radioButton = QtWidgets.QRadioButton(self.centralwidget)
        self.radioButton.setGeometry(QtCore.QRect(420, 40, 131, 17))
        self.radioButton.setObjectName("radioButton")
        self.radioButton_2 = QtWidgets.QRadioButton(self.centralwidget)
        self.radioButton_2.setGeometry(QtCore.QRect(420, 70, 361, 17))
        self.radioButton_2.setObjectName("radioButton_2")
        self.radioButton_4 = QtWidgets.QRadioButton(self.centralwidget)
        self.radioButton_4.setGeometry(QtCore.QRect(420, 100, 361, 16))
        self.radioButton_4.setObjectName("radioButton_4")
        self.radioButton_5 = QtWidgets.QRadioButton(self.centralwidget)
        self.radioButton_5.setGeometry(QtCore.QRect(420, 130, 361, 16))
        self.radioButton_5.setObjectName("radioButton_5")
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(40, 25, 341, 23))
        self.pushButton.setObjectName("pushButton")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(40, 65, 360, 180))
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(40, 105, 341, 21))
        self.label_2.setObjectName("label_2")
        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setGeometry(QtCore.QRect(420, 20, 191, 16))
        self.label_3.setObjectName("label_3")
        self.pushButton_2 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_2.setGeometry(QtCore.QRect(420, 160, 361, 23))
        self.pushButton_2.setObjectName("pushButton_2")
        self.pushButton_3 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_3.setGeometry(QtCore.QRect(420, 190, 361, 23))
        self.pushButton_3.setObjectName("pushButton_3")
        self.pushButton_5 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_5.setGeometry(QtCore.QRect(40, 55, 341, 23))
        self.pushButton_5.setObjectName("pushButton5")
        self.pushButton_6 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_6.setGeometry(QtCore.QRect(40, 85, 341, 23))
        self.pushButton_6.setObjectName("pushButton5")
        MainWindow.setCentralWidget(self.centralwidget)
        self.pushButton.clicked.connect(self.filepath)
        self.retranslateUi(MainWindow)
        self.pushButton_6.clicked.connect(self.parametrs)
        self.pushButton_3.clicked.connect(self.openNewWindow)
        self.pushButton_2.clicked.connect(self.alg)
        self.pushButton_5.clicked.connect(self.rand)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def parametrs(self):
        self.N, ok2 = QInputDialog.getInt(self.centralwidget, 'Поле вводу', 'Введіть кількість ітерацій:')
        if not ok2 or self.N == 0:
            self.N = 50

    def rand(self):
        self.n, ok1 = QInputDialog.getInt(self.centralwidget, 'Поле вводу', 'Введіть кількість партій в задачі:')
        if not ok1 or self.n in range(0,5):
            self.n = random.randint(5,8)
        self.label_2.setText("Рандомізована задача:")
        self.SumDep = 0
        uzg.clear()
        koldep.clear()
        d1 = 0.1
        d2 = 35
        self.SumDep,self.koldep = GenerTask(self.n,d1,d2)
        self.openMatrix()
        self.label.setText("Кількість фракцій: " + str(self.n) 
                           + "\nКількість депутатів в кожній фракції:\n" 
                           + str(self.koldep))
    def alg(self):
        if self.radioButton_4.isChecked():
            rez = QMessageBox()
            rez.setWindowTitle("Розв'язання задачі жадібним алгоритмом")
            a,p,c = jadniyalg(self.n,self.SumDep,PrUzg,self.koldep)
            for i in range(len(p)):
                p[i] += 1
            reztext = "Значення рекорду: " + str(a) + "\nПартії, що утворюють коаліцію: " + str(p) + "\nСума депутатів в коаліції: " + str(c)
            rez.setText(reztext)
            rez.setStandardButtons(QMessageBox.Ok)
            rez.exec_()
        if self.radioButton_2.isChecked():
            rez = QMessageBox()
            rez.setWindowTitle("Розв'язання задачі генетичним алгоритмом з оператором інбридинг")
            a,p,c = GA(self.n,self.SumDep,PrUzg,1,self.N,self.koldep)
            for i in range(len(p)):
                p[i] += 1
            reztext = "Значення рекорду: " + str(a) + "\nПартії, що утворюють коаліцію: " + str(p) + "\nСума депутатів в коаліції: " + str(c)
            rez.setText(reztext)
            rez.setStandardButtons(QMessageBox.Ok)
            rez.exec_()
        if self.radioButton.isChecked():
            rez = QMessageBox()
            rez.setWindowTitle("Розв'язання задачі методом гілок та меж")
            a,p,c = mgtmalg(self.n,self.SumDep,PrUzg,self.koldep)
            reztext = "Значення рекорду: " + str(a) + "\nПартії, що утворюють коаліцію: " + str(p) + "\nСума депутатів в коаліції: " + str(c)
            rez.setText(reztext)
            rez.setStandardButtons(QMessageBox.Ok)
            rez.exec_()
        if self.radioButton_5.isChecked():
            rez = QMessageBox()
            rez.setWindowTitle("Розв'язання задачі трьома алгоритмами")
            reztext = ""
            a,p,c = jadniyalg(self.n,self.SumDep,PrUzg,self.koldep)
            for i in range(len(p)):
                p[i] += 1
            reztext += "Жадібним алгоритмом\nЗначення рекорду: " + str(a) + "\nПартії, що утворюють коаліцію: " + str(p) + "\nСума депутатів в коаліції: " + str(c)
            b,w,g = GA(self.n,self.SumDep,PrUzg,1,self.N,self.koldep)
            for i in range(len(w)):
                w[i] += 1
            reztext += "\n\nГенетичним алгоритмом з оператором інбридінгу\nЗначення рекорду: " + str(b) + "\nПартії, що утворюють коаліцію: " + str(w) + "\nСума депутатів в коаліції: " + str(g)
            o,k,l = mgtmalg(self.n,self.SumDep,PrUzg,self.koldep)
            reztext += "\n\nМетод гілок та меж\nЗначення рекорду: " + str(o) + "\nПартії, що утворюють коаліцію: " + str(k) + "\nСума депутатів в коаліції: " + str(l)
            rez.setText(reztext)
            rez.setStandardButtons(QMessageBox.Ok)
            rez.exec_()
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Курсова робота | Бородай Катеринa та Максимчук Володимир | IS-82 | Дослідження операцій 2021"))
        self.radioButton.setText(_translate("MainWindow", "Метод гілок та меж"))
        self.radioButton_2.setText(_translate("MainWindow", "Генетичний алгоритм з оператором вибору батьків -  інбридинг"))
        self.radioButton_4.setText(_translate("MainWindow", "Жадібний алгоритм"))
        self.radioButton_5.setText(_translate("MainWindow", "Трьома алгоритмами одразу"))
        self.pushButton.setText(_translate("MainWindow", "Обрати файл з задачею"))
        self.pushButton_5.setText(_translate("MainWindow", "Рандомізувати задачу"))
        self.label.setText(_translate("MainWindow", ""))
        self.label_2.setText(_translate("MainWindow", ""))
        self.label_3.setText(_translate("MainWindow", "Розв'язати алгоритмом:"))
        self.pushButton_2.setText(_translate("MainWindow", "Розв'язати задачу"))
        self.pushButton_3.setText(_translate("MainWindow", "Перейти до експериментів"))
        self.pushButton_6.setText(_translate("MainWindow", "Задати кількість ітерацій"))
    def filepath(self):
        self.label_2.setText("Зчитана задача:")
        self.SumDep = 0
        uzg.clear()
        self.koldep.clear()
        a = QtWidgets.QFileDialog.getOpenFileName()[0]
        try:
            f = open(a, 'r')
            file = f.readlines()
            self.n = int(file[0].rstrip())
            koldepstr = file[1]
            self.koldep = koldepstr.split(",")
            self.koldep = [int(line.rstrip()) for line in self.koldep]
            for i in range(self.n):
                e = file[2+i].rstrip()
                uzg.append(e.split(','))
            for i in range(self.n):
                for j in range(self.n):
                    uzg[i][j] = float(uzg[i][j])
            for i in range(self.n):
                self.SumDep += self.koldep[i]
            self.openMatrix()
            self.label.setText("Кількість фракцій: " + str(self.n) + 
                               "\nКількість депутатів в кожній фракції:\n" 
                               + str(self.koldep))
        except: print()
    def openNewWindow(self):
        self.w = Window(self)
        self.w.show()
    def openMatrix(self):
        self.wi = WindowMatrix(self)
        self.wi.show()
        
class Window(QtWidgets.QMainWindow):
    def __init__(self, info_str, parent=None):
        super(Window, self).__init__(parent)
        self.setWindowTitle("Курсова робота | Бородай Катеринa та Максимчук Володимир | IS-82 | Дослідження операцій 2021")
        self.setGeometry(0,0,825,800)
        self.center()
        self.html = self.Experiments()
        self.plot_widget = QWebEngineView()
        self.plot_widget.setHtml(self.html)
        self.setCentralWidget(self.plot_widget)
    def Experiments(self):
        uzg.clear()
        koldep.clear()
        self.html = '<html><body>'
        PrUzg = 0.1 ######################################
        NIter = 10
        nn = []
        d1 = 0.1
        nstart = 10
        nend = 65
        nstep = 1
        d2 = 30
        N = 10
        iterator = 0
        genetich = [np.float64(np.zeros(nend-nstart)/nstep) for x in range(2)]
        for n in range(nstart,nend,nstep):
            nn.append(n)
            SumDep,w = GenerTask(n,d1,d2)
            for i in range(2):
                genetichiter = np.zeros(NIter)
                for j in range(NIter):
                    a,p,c = GA(n,SumDep,PrUzg,i+1,N,koldep)
                    genetichiter[j] = a
                genetich[i][iterator] = np.mean(genetichiter,dtype=np.float64)
            iterator+=1
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=genetich[0], x=nn, name='Генетичний з оператором інбридінгу',line=dict(color="#3EFE5E")))
        fig.add_trace(go.Scatter(y=genetich[1], x=nn, name='Генетичний з оператором аутбридінгу',line=dict(color="#AD3EFE")))
        fig.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №1",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        self.html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn')
        d1 = 0.1
        d2 = 35
        NIter = 10
        PrUzg = 0.5
        n=10
        operator = 1
        nstart = 10
        nend = 200
        nstep = 10
        nn = []
        iterator = 0
        genetichexp4 = [np.float64(np.zeros(nend-nstart)/nstep)]
        SumDep,w = GenerTask(n,d1,d2)
        for N in range(nstart,nend,nstep):
            nn.append(N)
            genetichiter = np.zeros(NIter)
            for i in range(NIter):
                a,p,c = GA(n,SumDep,PrUzg,operator,N,koldep)
                genetichiter[i] = a
            genetichexp4[0][iterator] = np.mean(genetichiter,dtype=np.float64)
            iterator+=1
        fig = go.Figure()
        fig.add_trace(go.Scatter(y=genetichexp4[0], x=nn, name='Генетичний алгоритм',line=dict(color="#FF9A21")))
        fig.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №2",
                          xaxis_title="Кількість послідовних ітерацій, впродовж яких рекордне значення ЦФ не змінюється",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        self.html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn')
        d2 = 30
        N=50
        NIter = 10
        PrUzg = 0.15
        nstart = 10
        nend = 65
        nstep = 1
        jadniydis = [np.float64(np.zeros(nend-nstart)/nstep) for x in range(2)]
        genetichdis = [np.float64(np.zeros(nend-nstart)/nstep) for x in range(2)]
        dd = [0.1,0.4]
        iterator = 0
        nn = []
        operator = 1
        for n in range(10,65,1):
            nn.append(n)
            for i in range(2):
                d1 = dd[i]
                SumDep,w = GenerTask(n,d1,d2)
                jadniyiter = np.zeros(NIter)
                genetichiter = np.zeros(NIter)
                for j in range(NIter):
                    a,p,c = jadniyalg(n,SumDep,PrUzg,koldep)
                    b,d,f = GA(n,SumDep,PrUzg,operator,N,koldep)
                    jadniyiter[j] = a
                    genetichiter[j] = b
                jadniydis[i][iterator] = np.mean(jadniyiter,dtype=np.float64)
                genetichdis[i][iterator] = np.mean(genetichiter,dtype=np.float64)
            iterator+=1
        fig = go.Figure()
        fig2 = go.Figure()
        fig.add_trace(go.Scatter(y=jadniydis[0], x=nn, name='Жадібний з d1',line=dict(color="#FE3E3E")))
        fig.add_trace(go.Scatter(y=jadniydis[1], x=nn, name='Жадібний з d2',line=dict(color="#3E8AFE")))
        fig2.add_trace(go.Scatter(y=genetichdis[0], x=nn, name='Генетичний з d1',line=dict(color="#3EFE5E")))
        fig2.add_trace(go.Scatter(y=genetichdis[1], x=nn, name='Генетичний з d2',line=dict(color="#AD3EFE")))
        fig.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №3 | Жадібний алгоритм",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        fig2.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №3 | Генетичний алгоритм",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        self.html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn')
        self.html += plotly.offline.plot(fig2, output_type='div', include_plotlyjs='cdn')
        NIter = 10
        operator = 1
        PrUzg = 0.05 ###########################################
        N = 50
        nstart = 10
        nend = 65
        nstep = 1
        jadniydis = [np.float64(np.zeros(nend-nstart)/nstep) for x in range(2)]
        genetichdis = [np.float64(np.zeros(nend-nstart)/nstep) for x in range(2)]
        d1 = 0.4
        dd = [20,50]
        iterator = 0
        nn = []
        for n in range(10,65,1):
            nn.append(n)
            for i in range(2):
                d2 = dd[i]
                SumDep,w = GenerTask(n,d1,d2)
                jadniyiter = np.zeros(NIter)
                genetichiter = np.zeros(NIter)
                for j in range(NIter):
                    a,b,c = jadniyalg(n,SumDep,PrUzg,koldep)
                    e,f,p = GA(n,SumDep,PrUzg,operator,N,koldep)
                    jadniyiter[j] = a
                    genetichiter[j] = e
                jadniydis[i][iterator] = np.mean(jadniyiter,dtype=np.float64)
                genetichdis[i][iterator] = np.mean(genetichiter,dtype=np.float64)
            iterator+=1
        fig = go.Figure()
        fig2 = go.Figure()
        fig.add_trace(go.Scatter(y=jadniydis[0], x=nn, name='Жадібний з d1',line=dict(color="#FE3E3E")))
        fig.add_trace(go.Scatter(y=jadniydis[1], x=nn, name='Жадібний з d2',line=dict(color="#3E8AFE")))
        fig2.add_trace(go.Scatter(y=genetichdis[0], x=nn, name='Генетичний з d1',line=dict(color="#3EFE5E")))
        fig2.add_trace(go.Scatter(y=genetichdis[1], x=nn, name='Генетичний з d2',line=dict(color="#AD3EFE")))
        fig.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №4 | Жадібний алгоритм",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        fig2.update_layout(legend_orientation="h",
                          legend=dict(x=.5, xanchor="center"),
                          title="Експеримент №4 | Генетичний алгоритм",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=750,
                          width=800)
        self.html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn')
        self.html += plotly.offline.plot(fig2, output_type='div', include_plotlyjs='cdn')
        d1 = 0.4
        d2 = 50
        NIter = 10
        nstart = 5
        nend = 9
        nstep = 1
        PrUzg = 0.5
        operator = 1
        N = 50
        iterator = 0
        nn = []
        mgtm = [np.float64(np.zeros(nend-nstart)/nstep)]
        jadniy = [np.float64(np.zeros(nend-nstart)/nstep)]
        genetich = [np.float64(np.zeros(nend-nstart)/nstep)]
        jadniyotkl = [np.float64(np.zeros(nend-nstart)/nstep)]
        genetichotkl = [np.float64(np.zeros(nend-nstart)/nstep)]
        for n in range(nstart,nend,nstep):
            nn.append(n)
            SumDep,w = GenerTask(n,d1,d2)
            genetichiter = np.zeros(NIter)
            jadniyiter = np.zeros(NIter)
            mgtmiter = np.zeros(NIter)
            for i in range(NIter):
                a,b,c = GA(n,SumDep,PrUzg,operator,N,koldep)
                e,f,p = jadniyalg(n,SumDep,PrUzg,koldep)
                h,k,o = mgtmalg(n,SumDep,PrUzg,koldep)
                genetichiter[i] = a
                jadniyiter[i] = e
                mgtmiter[i] = h
            genetich[0][iterator] = np.mean(genetichiter,dtype=np.float64)
            jadniy[0][iterator] = np.mean(jadniyiter,dtype=np.float64)
            mgtm[0][iterator] = np.mean(mgtmiter,dtype=np.float64)
            w = abs((genetich[0][iterator]-mgtm[0][iterator])/mgtm[0][iterator])
            l = abs((jadniy[0][iterator]-mgtm[0][iterator])/mgtm[0][iterator])
            genetichotkl[0][iterator] = w*100
            jadniyotkl[0][iterator] = l*100
            iterator+=1
        fig = go.Figure()
        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(y=jadniyotkl[0], x=nn, name='Жадібний алгоритм',line=dict(color="#3E8AFE")))
        fig2.add_trace(go.Scatter(y=genetichotkl[0], x=nn, name='Генетичний алгоритм',line=dict(color="#3EFE5E")))
        fig.add_trace(go.Scatter(y=jadniy[0], x=nn, name='Жадібний алгоритм',line=dict(color="#3E8AFE")))
        fig.add_trace(go.Scatter(y=genetich[0], x=nn, name='Генетичний алгоритм',line=dict(color="#3EFE5E")))
        fig.add_trace(go.Scatter(y=mgtm[0], x=nn, name='Метод гілок та меж',line=dict(color="#FE3E3E")))
        fig.update_layout(legend_orientation="h",
                          title="Експеримент №5 | Порівняння результатів",
                          xaxis_title="Кількість партій",
                          yaxis_title="Значення рекорду",
                          height=800,
                          width=800,
                          legend=dict(x=.5, xanchor="center"))
        fig2.update_layout(legend_orientation="h",
                          title="Експеримент №5 | Порівняння похибок",
                          xaxis_title="Кількість партій",
                          yaxis_title="%",
                          height=800,
                          width=800,
                          legend=dict(x=.5, xanchor="center"))
        self.html += plotly.offline.plot(fig, output_type='div', include_plotlyjs='cdn')
        self.html += plotly.offline.plot(fig2, output_type='div', include_plotlyjs='cdn')
        self.html += '</body></html>'
        return self.html
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())

class WindowMatrix(QtWidgets.QMainWindow):
    def __init__(self, info_str, parent=None):
        super(WindowMatrix, self).__init__(parent)
        self.setWindowTitle("Матриця узгодженностей")
        self.setGeometry(0,0,330,270)
        self.center()
        self.html2 = self.df()
        self.plot_widget1 = QWebEngineView()
        self.plot_widget1.setHtml(self.html2)
        self.setCentralWidget(self.plot_widget1)
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
    def df(self):
        df = p.DataFrame(data=uzg)
        self.html2 = '<html><body>'
        self.html2 = df.to_html()
        self.html2 += '</body></html>'
        return self.html2

def application():
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())
application()

SystemExit: 0

f:\python 3.8.2\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

